In [1]:
import math
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)


In [2]:
titles = [
    # SF Masterworks (horror / horror-adjacent / dark SF)
    {"series":"SF Masterworks", "title":"I Am Legend", "author":"Richard Matheson",
     "subgenre":"vampiric apocalypse", "horror_adjacent":True, "format":"paperback", "rrp_gbp":10.99},
    {"series":"SF Masterworks", "title":"Hyperion", "author":"Dan Simmons",
     "subgenre":"dark space opera", "horror_adjacent":True, "format":"paperback", "rrp_gbp":10.99},
    {"series":"SF Masterworks", "title":"The Fall of Hyperion", "author":"Dan Simmons",
     "subgenre":"dark space opera", "horror_adjacent":True, "format":"paperback", "rrp_gbp":10.99},
    {"series":"SF Masterworks", "title":"Dune", "author":"Frank Herbert",
     "subgenre":"epic SF", "horror_adjacent":False, "format":"paperback", "rrp_gbp":12.99},
    {"series":"SF Masterworks", "title":"The Dispossessed", "author":"Ursula K. Le Guin",
     "subgenre":"social SF", "horror_adjacent":False, "format":"paperback", "rrp_gbp":10.99},
    {"series":"SF Masterworks", "title":"Revelation Space", "author":"Alastair Reynolds",
     "subgenre":"hard SF / gothic space", "horror_adjacent":True, "format":"paperback", "rrp_gbp":14.99},
    
    # OPTIONAL: add “non-SF Masterworks” classic horror comparators (useful for a horror dataset)
    {"series":"Classic Horror Comparator", "title":"Dracula", "author":"Bram Stoker",
     "subgenre":"gothic horror", "horror_adjacent":True, "format":"paperback", "rrp_gbp":8.99},
    {"series":"Classic Horror Comparator", "title":"Frankenstein", "author":"Mary Shelley",
     "subgenre":"gothic horror", "horror_adjacent":True, "format":"paperback", "rrp_gbp":7.99},
    {"series":"Classic Horror Comparator", "title":"The Haunting of Hill House", "author":"Shirley Jackson",
     "subgenre":"haunted house", "horror_adjacent":True, "format":"paperback", "rrp_gbp":9.99},
]

df_books = pd.DataFrame(titles)
df_books


,series,title,author,subgenre,horror_adjacent,format,rrp_gbp
0,SF Masterworks,I Am Legend,Richard Matheson,vampiric apocalypse,True,paperback,10.99
1,SF Masterworks,Hyperion,Dan Simmons,dark space opera,True,paperback,10.99
2,SF Masterworks,The Fall of Hyperion,Dan Simmons,dark space opera,True,paperback,10.99
3,SF Masterworks,Dune,Frank Herbert,epic SF,False,paperback,12.99
4,SF Masterworks,The Dispossessed,Ursula K. Le Guin,social SF,False,paperback,10.99
5,SF Masterworks,Revelation Space,Alastair Reynolds,hard SF / gothic space,True,paperback,14.99
6,Classic Horror Comparator,Dracula,Bram Stoker,gothic horror,True,paperback,8.99
7,Classic Horror Comparator,Frankenstein,Mary Shelley,gothic horror,True,paperback,7.99
8,Classic Horror Comparator,The Haunting of Hill House,Shirley Jackson,haunted house,True,paperback,9.99


In [3]:
retailers = pd.DataFrame([
    {
        "retailer":"Waterstones",
        "channel":"High street bookshop",
        "positioning":"Curated specialist bookseller (destination browsing)",
        "primary_missions":"discovery, leisure reading, gifting",
        # assumptions (edit freely)
        "avg_discount_to_customer":0.10,          # e.g., promos/loyalty
        "gross_margin_books":0.35,                # adjust: UK book margins are pressured/low in aggregate :contentReference[oaicite:7]{index=7}
        "avg_basket_units":1.3,
        "avg_addon_rate":0.12,                    # e.g., add a 2nd book
        "footfall_per_store_per_week":4500,       # typical-ish high-street (assumption)
        "conversion_rate":0.085,                  # browse-to-buy
        "horror_share_of_book_units":0.10,        # share of unit sales for horror/dark SF
    },
    {
        "retailer":"WHSmith Travel",
        "channel":"Travel retail (air/rail/hospital)",
        "positioning":"Convenience + impulse travel essentials",
        "primary_missions":"impulse reading, travel comfort, gifts/snacks/stationery",
        # assumptions (edit freely)
        "avg_discount_to_customer":0.05,          # less discounting typical in captive locations (assumption)
        "gross_margin_books":0.30,                # book margin may be lower vs non-book; total basket margin could be higher with essentials
        "avg_basket_units":1.05,
        "avg_addon_rate":0.05,
        "footfall_per_store_per_week":12000,      # travel footfall (assumption)
        "conversion_rate":0.045,                  # lower conversion (many passersby)
        "horror_share_of_book_units":0.06,        # more mainstream/seasonal in travel (assumption)
    }
])

retailers


,retailer,channel,positioning,primary_missions,avg_discount_to_customer,gross_margin_books,avg_basket_units,avg_addon_rate,footfall_per_store_per_week,conversion_rate,horror_share_of_book_units
0,Waterstones,High street bookshop,Curated specialist bookseller (destination bro...,"discovery, leisure reading, gifting",0.10,0.35,1.30,0.12,4500,0.085,0.10
1,WHSmith Travel,Travel retail (air/rail/hospital),Convenience + impulse travel essentials,"impulse reading, travel comfort, gifts/snacks/...",0.05,0.30,1.05,0.05,12000,0.045,0.06


In [4]:
df_horror = df_books[df_books["horror_adjacent"]].copy()
df_horror


,series,title,author,subgenre,horror_adjacent,format,rrp_gbp
0,SF Masterworks,I Am Legend,Richard Matheson,vampiric apocalypse,True,paperback,10.99
1,SF Masterworks,Hyperion,Dan Simmons,dark space opera,True,paperback,10.99
2,SF Masterworks,The Fall of Hyperion,Dan Simmons,dark space opera,True,paperback,10.99
5,SF Masterworks,Revelation Space,Alastair Reynolds,hard SF / gothic space,True,paperback,14.99
6,Classic Horror Comparator,Dracula,Bram Stoker,gothic horror,True,paperback,8.99
7,Classic Horror Comparator,Frankenstein,Mary Shelley,gothic horror,True,paperback,7.99
8,Classic Horror Comparator,The Haunting of Hill House,Shirley Jackson,haunted house,True,paperback,9.99


In [5]:
# --- user inputs (edit these) ---
stores_in_england = {
    "Waterstones": 220,      # placeholder; set to your estimate
    "WHSmith Travel": 300    # placeholder; set to your estimate (air/rail/hospital sites in England)
}

WEEKS_PER_YEAR = 52

def expected_units_sold_per_week(row):
    buyers = row["footfall_per_store_per_week"] * row["conversion_rate"]
    units = buyers * row["avg_basket_units"] * (1 + row["avg_addon_rate"])
    return units

def simulate_week(retailer_row, books_df, rng, n_customers_noise=True):
    """
    Simulate one week of book sales for a single store.
    Returns revenue, gross_profit, units, and units of horror-adjacent.
    """
    footfall = retailer_row["footfall_per_store_per_week"]
    conv = retailer_row["conversion_rate"]
    
    # stochastic buyers
    if n_customers_noise:
        buyers = rng.binomial(int(footfall), conv)
    else:
        buyers = int(round(footfall * conv))
    
    # units per buyer with mild noise
    avg_units = retailer_row["avg_basket_units"]
    units = rng.poisson(lam=max(0.1, buyers * avg_units))
    
    # how much of those units are horror-adjacent
    horror_units = rng.binomial(int(max(0, units)), retailer_row["horror_share_of_book_units"])
    
    # choose titles for horror units; assume mix across list
    if len(books_df) == 0:
        return 0.0, 0.0, 0, 0
    
    chosen = books_df.sample(n=int(horror_units), replace=True, random_state=rng.integers(0, 1_000_000))
    avg_discount = retailer_row["avg_discount_to_customer"]
    sell_prices = chosen["rrp_gbp"] * (1 - avg_discount)
    revenue = sell_prices.sum()
    
    # gross profit on books
    gm = retailer_row["gross_margin_books"]
    gross_profit = revenue * gm
    
    return float(revenue), float(gross_profit), int(units), int(horror_units)

def run_simulation(retailers_df, books_df, stores_map, weeks=52, sims=2000, seed=7):
    rng = np.random.default_rng(seed)
    rows = []
    for _, r in retailers_df.iterrows():
        name = r["retailer"]
        n_stores = stores_map.get(name, 0)
        if n_stores <= 0:
            continue
        
        for s in range(sims):
            weekly_revenue_store, weekly_gp_store, weekly_units_store, weekly_horror_units_store = simulate_week(r, books_df, rng)
            
            # scale to England-wide and to annual
            weekly_revenue_eng = weekly_revenue_store * n_stores
            weekly_gp_eng = weekly_gp_store * n_stores
            
            annual_revenue_eng = weekly_revenue_eng * weeks
            annual_gp_eng = weekly_gp_eng * weeks
            
            rows.append({
                "retailer": name,
                "stores_in_england": n_stores,
                "weekly_revenue_per_store": weekly_revenue_store,
                "weekly_gross_profit_per_store": weekly_gp_store,
                "weekly_revenue_england": weekly_revenue_eng,
                "weekly_gross_profit_england": weekly_gp_eng,
                "annual_revenue_england": annual_revenue_eng,
                "annual_gross_profit_england": annual_gp_eng,
                "weekly_units_per_store": weekly_units_store,
                "weekly_horror_units_per_store": weekly_horror_units_store,
            })
    return pd.DataFrame(rows)

sim = run_simulation(retailers, df_horror, stores_in_england, weeks=WEEKS_PER_YEAR, sims=5000, seed=42)
sim.head()




,retailer,stores_in_england,weekly_revenue_per_store,weekly_gross_profit_per_store,weekly_revenue_england,weekly_gross_profit_england,annual_revenue_england,annual_gross_profit_england,weekly_units_per_store,weekly_horror_units_per_store
0,Waterstones,220,415.413,145.39455,91390.86,31986.801,4752324.72,1663313.652,511,43
1,Waterstones,220,591.651,207.07785,130163.22,45557.127,6768487.44,2368970.604,486,61
2,Waterstones,220,402.831,140.99085,88622.82,31017.987,4608386.64,1612935.324,484,41
3,Waterstones,220,377.640,132.17400,83080.80,29078.280,4320201.60,1512070.560,497,40
4,Waterstones,220,365.967,128.08845,80512.74,28179.459,4186662.48,1465331.868,463,37


In [6]:
def summarise(sim_df):
    def q(x, p): 
        return float(np.quantile(x, p))
    
    out = []
    for name, g in sim_df.groupby("retailer"):
        out.append({
            "retailer": name,
            "stores_in_england": int(g["stores_in_england"].iloc[0]),
            "weekly_revenue_eng_median": q(g["weekly_revenue_england"], 0.50),
            "weekly_revenue_eng_p10": q(g["weekly_revenue_england"], 0.10),
            "weekly_revenue_eng_p90": q(g["weekly_revenue_england"], 0.90),
            "weekly_gross_profit_eng_median": q(g["weekly_gross_profit_england"], 0.50),
            "annual_revenue_eng_median": q(g["annual_revenue_england"], 0.50),
            "annual_gross_profit_eng_median": q(g["annual_gross_profit_england"], 0.50),
            "weekly_horror_units_per_store_median": q(g["weekly_horror_units_per_store"], 0.50),
        })
    return pd.DataFrame(out).sort_values("annual_gross_profit_eng_median", ascending=False)

summary = summarise(sim)
summary


,retailer,stores_in_england,weekly_revenue_eng_median,weekly_revenue_eng_p10,weekly_revenue_eng_p90,weekly_gross_profit_eng_median,annual_revenue_eng_median,annual_gross_profit_eng_median,weekly_horror_units_per_store_median
1,Waterstones,220,104644.98,84837.852,125811.180,36625.743,5441538.96,1904538.636,49.0
0,WHSmith Travel,300,103640.25,80008.050,127560.585,31092.075,5389293.00,1616787.900,34.0


In [7]:
logic = retailers[[
    "retailer","channel","positioning","primary_missions",
    "footfall_per_store_per_week","conversion_rate","avg_basket_units",
    "avg_discount_to_customer","gross_margin_books","horror_share_of_book_units"
]].copy()

logic


,retailer,channel,positioning,primary_missions,footfall_per_store_per_week,conversion_rate,avg_basket_units,avg_discount_to_customer,gross_margin_books,horror_share_of_book_units
0,Waterstones,High street bookshop,Curated specialist bookseller (destination bro...,"discovery, leisure reading, gifting",4500,0.085,1.30,0.10,0.35,0.10
1,WHSmith Travel,Travel retail (air/rail/hospital),Convenience + impulse travel essentials,"impulse reading, travel comfort, gifts/snacks/...",12000,0.045,1.05,0.05,0.30,0.06
